In [ ]:
# !pip install simple_salesforce
# !pip install pymysql
# !pip install sqlalchemy

In [86]:
import json
from sqlalchemy import create_engine
from datetime import date
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

In [87]:
# Make sure to use your own `config.py` file. Consider ensuring that these variable names are in sync
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [88]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [89]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

Exception during reset or similar
Traceback (most recent call last):
  File "C:\Users\mahdi\Anaconda3\lib\site-packages\pymysql\connections.py", line 683, in _read_bytes
    data = self._rfile.read(num_bytes)
  File "C:\Users\mahdi\Anaconda3\lib\socket.py", line 669, in readinto
    return self._sock.recv_into(b)
TimeoutError: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\mahdi\Anaconda3\lib\site-packages\sqlalchemy\pool\base.py", line 697, in _finalize_fairy
    fairy._reset(pool)
  File "C:\Users\mahdi\Anaconda3\lib\site-packages\sqlalchemy\pool\base.py", line 893, in _reset
    pool._dialect.do_rollback(self)
  File "C:\Users\mahdi\Anaconda3\lib\site-packages\sqlalchemy\dialects\mysql\base.py", line 2374, in do_

## Prepare ETL for the Course data

In [9]:
course_data_df = pd.read_sql("SELECT * FROM course", conn)
course_data_df.head(30)

,ID_Course,CourseCode,CourseName,CreditHours,BootCampCourse
0,1,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,2,BC-WEBDEV,Full Stack Web Development,12,1
2,3,BC-UIUX,User Interface/User Experience,12,1
3,4,CIS-349,Introduction to Databases,5,0
4,5,CIS-405,Database Programming,5,0
5,6,CIS-438,Database Administration,5,0
6,7,CIS-430,Business Systems Programming I,5,0
7,8,CIS-435,Business Systems Programming II,5,0


In [10]:
course_data_df.rename(columns={
    'CourseCode':'Course_Code__c',
    'CourseName':'Course_Name__c',
    'CreditHours':'Credit_Hours__c',
    'BootCampCourse':'Boot_Camp_Course__c',    
}, inplace=True)
course_data_df.head()

,ID_Course,Course_Code__c,Course_Name__c,Credit_Hours__c,Boot_Camp_Course__c
0,1,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,2,BC-WEBDEV,Full Stack Web Development,12,1
2,3,BC-UIUX,User Interface/User Experience,12,1
3,4,CIS-349,Introduction to Databases,5,0
4,5,CIS-405,Database Programming,5,0


In [11]:
course_data_df = course_data_df[['Course_Code__c', 'Course_Name__c', 'Credit_Hours__c', 'Boot_Camp_Course__c']]
course_data_df.head()

,Course_Code__c,Course_Name__c,Credit_Hours__c,Boot_Camp_Course__c
0,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,BC-WEBDEV,Full Stack Web Development,12,1
2,BC-UIUX,User Interface/User Experience,12,1
3,CIS-349,Introduction to Databases,5,0
4,CIS-405,Database Programming,5,0


In [12]:
course_data_records = course_data_df.to_dict('records')
course_data_records

[{'Course_Code__c': 'BC-DATAVIZ',
  'Course_Name__c': 'Data Visualization and Analytics',
  'Credit_Hours__c': 12,
  'Boot_Camp_Course__c': 1},
 {'Course_Code__c': 'BC-WEBDEV',
  'Course_Name__c': 'Full Stack Web Development',
  'Credit_Hours__c': 12,
  'Boot_Camp_Course__c': 1},
 {'Course_Code__c': 'BC-UIUX',
  'Course_Name__c': 'User Interface/User Experience',
  'Credit_Hours__c': 12,
  'Boot_Camp_Course__c': 1},
 {'Course_Code__c': 'CIS-349',
  'Course_Name__c': 'Introduction to Databases',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'Course_Code__c': 'CIS-405',
  'Course_Name__c': 'Database Programming',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'Course_Code__c': 'CIS-438',
  'Course_Name__c': 'Database Administration',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'Course_Code__c': 'CIS-430',
  'Course_Name__c': 'Business Systems Programming I',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'Course_Code__c': 'CIS-435',
  'Course_Name__c':

In [13]:
for rec in course_data_records:

    record = {
        'Course_Code__c': rec['Course_Code__c'],
        'Course_Name__c': rec['Course_Name__c'],
        'Credit_Hours__c': rec['Credit_Hours__c'],
        'Boot_Camp_Course__c': rec['Boot_Camp_Course__c'],
    }
    
    try:
        sf.Course__c.create(record)
    except Exception as e:
        print(e)

In [ ]:
# Bulk 
#sf.bulk.Course__c.insert(course_data_records)

# Prepare ETL for the Student data


In [15]:
student_data_df = pd.read_sql("SELECT * FROM student", conn)
student_data_df.head()

,ID_Student,StudentID,LastName,FirstName,MiddleName,BirthDate,Gender
0,33,25004961,Dartling,Heather,Alice,None,F
1,34,25003514,Dartling,Lana,Cecille,None,F
2,35,25005833,Dartling,Jessica,Dorothy,None,F
3,36,25002589,Dartling,Kimberly,Genevieve,None,F
4,37,25007185,Dartling,Katherine,Lynnette,None,F


In [28]:
#rename columns from student table
student_data_df.rename(columns={
    'StudentID':'Student_Record_ID__c',
    'LastName':'Last_Name__c',
    'FirstName':'First_Name__c',
    'MiddleName':'Middle_Name__c', 
    'BirthDate':'Birth_Date__c',
    'Gender': 'Gender__c',
}, inplace=True)

student_data_df

#Dropping ID_Student
student_data_df = student_data_df[['Student_Record_ID__c', 'Last_Name__c', 'First_Name__c', 'Middle_Name__c', 'Birth_Date__c', 'Gender__c']]
student_data_df.head()

,Student_Record_ID__c,Last_Name__c,First_Name__c,Middle_Name__c,Birth_Date__c,Gender__c
0,25004961,Dartling,Heather,Alice,None,F
1,25003514,Dartling,Lana,Cecille,None,F
2,25005833,Dartling,Jessica,Dorothy,None,F
3,25002589,Dartling,Kimberly,Genevieve,None,F
4,25007185,Dartling,Katherine,Lynnette,None,F


In [29]:
student_data_records = student_data_df.to_dict('records')
student_data_records

[{'Student_Record_ID__c': '25004961',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Heather',
  'Middle_Name__c': 'Alice',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'Student_Record_ID__c': '25003514',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Lana',
  'Middle_Name__c': 'Cecille',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'Student_Record_ID__c': '25005833',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Jessica',
  'Middle_Name__c': 'Dorothy',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'Student_Record_ID__c': '25002589',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Kimberly',
  'Middle_Name__c': 'Genevieve',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'Student_Record_ID__c': '25007185',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Katherine',
  'Middle_Name__c': 'Lynnette',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'Student_Record_ID__c': '25006014',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Precious',
  'Middle_Name__c': 'Mari

In [32]:
for rec in student_data_records:
    #taking a key out of dictionary putting into new dic format
    record = {
        'Student_ID__c':rec['Student_Record_ID__c'],
        'Last_Name__c':rec['Last_Name__c'],
        'First_Name__c':rec['First_Name__c'],
        'Middle_Name__c':rec['Middle_Name__c'], 
        'Birth_Date__c':rec['Birth_Date__c'],
        'Gender__c': rec['Gender__c'],
    }

    
    try:
        sf.Student__c.create(record)
    except Exception as e:
        print(e)

# Prepare ETL for the Staff data

In [17]:
staff_data_df = pd.read_sql("SELECT * FROM staff", conn)
staff_data_df.head()

,ID_Staff,EmployeeID,LastName,FirstName,MiddleName,BirthDate
0,1,000184220,Luongo,Darick,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Rodney,Heather,Nicole,1994-06-07


In [18]:
staff_data_df.rename(columns={
    'EmployeeID':'Employee_ID__c',
    'LastName':'Last_Name__c',
    'FirstName':'First_Name__c',
    'MiddleName':'Middle_Name__c', 
    'BirthDate':'Birth_Date__c',
}, inplace=True)

staff_data_df = staff_data_df[['Employee_ID__c', 'Last_Name__c', 'First_Name__c', 'Middle_Name__c', 'Birth_Date__c']]
staff_data_df.head()

,Employee_ID__c,Last_Name__c,First_Name__c,Middle_Name__c,Birth_Date__c
0,000184220,Luongo,Darick,Nico,1995-07-05
1,000130109,Sanford,Gemini,Blair,1992-04-22
2,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,000159108,Rodney,Heather,Nicole,1994-06-07


In [19]:
staff_data_records = staff_data_df.to_dict('records')
staff_data_records

[{'Employee_ID__c': '000184220',
  'Last_Name__c': 'Luongo',
  'First_Name__c': 'Darick',
  'Middle_Name__c': 'Nico',
  'Birth_Date__c': datetime.date(1995, 7, 5)},
 {'Employee_ID__c': '000130109',
  'Last_Name__c': 'Sanford',
  'First_Name__c': 'Gemini',
  'Middle_Name__c': 'Blair',
  'Birth_Date__c': datetime.date(1992, 4, 22)},
 {'Employee_ID__c': '000160655',
  'Last_Name__c': 'Williams',
  'First_Name__c': 'Dartanion',
  'Middle_Name__c': 'De Angelo',
  'Birth_Date__c': datetime.date(1993, 5, 21)},
 {'Employee_ID__c': '000159108',
  'Last_Name__c': 'Rodney',
  'First_Name__c': 'Heather',
  'Middle_Name__c': 'Nicole',
  'Birth_Date__c': datetime.date(1994, 6, 7)}]

In [20]:
for rec in staff_data_records:
   
    record = {
        'Employee_ID__c':rec['Employee_ID__c'],
        'Last_Name__c':rec['Last_Name__c'],
        'First_Name__c':rec['First_Name__c'],
        'Middle_Name__c':rec['Middle_Name__c'], 
        'Birth_Date__c':str(rec['Birth_Date__c'])
    } 
    try:
        sf.Staff__c.create(record)
    except Exception as e:
        print(e)

('Connection aborted.', TimeoutError(10060, 'A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond', None, 10060, None))


## Create Course Lookup Table
You will use this later to crosswalk the course code with the primary key from the `Course` table

It is important to note that we will be querying **Salesforce** to retrieve the record IDs 

In [21]:
course_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
# The Salesforce query language is called SOQL 
data = sf.query_all_iter("SELECT Name, Course_Code__c FROM Course__c")
for row in data:
    rec = {
        'ID_Course__c': row['Name'], # this is a critical line of code
        'Course_Code__c': row['Course_Code__c']
    }
    course_lookup_list.append(rec)
    
course_lookup_list

[{'ID_Course__c': 'a004x000003VQ9I', 'Course_Code__c': 'CIS-349'},
 {'ID_Course__c': 'a004x000003VQ9S', 'Course_Code__c': 'CIS-438'},
 {'ID_Course__c': 'a004x000003VQ98', 'Course_Code__c': 'BC-WEBDEV'},
 {'ID_Course__c': 'a004x000003VQ93', 'Course_Code__c': 'BC-DATAVIZ'},
 {'ID_Course__c': 'a004x000003VQ9X', 'Course_Code__c': 'CIS-430'},
 {'ID_Course__c': 'a004x000003VQ9c', 'Course_Code__c': 'CIS-435'},
 {'ID_Course__c': 'a004x000003VQ9N', 'Course_Code__c': 'CIS-405'},
 {'ID_Course__c': 'a004x000003UMcR', 'Course_Code__c': 'CIS-438'},
 {'ID_Course__c': 'a004x000003UMc2', 'Course_Code__c': 'BC-DATAVIZ'},
 {'ID_Course__c': 'a004x000003UMcW', 'Course_Code__c': 'CIS-430'},
 {'ID_Course__c': 'a004x000003UMcC', 'Course_Code__c': 'BC-UIUX'},
 {'ID_Course__c': 'a004x000003UMcb', 'Course_Code__c': 'CIS-435'},
 {'ID_Course__c': 'a004x000003UMcM', 'Course_Code__c': 'CIS-405'},
 {'ID_Course__c': 'a004x000003UMc7', 'Course_Code__c': 'BC-WEBDEV'},
 {'ID_Course__c': 'a004x000003UMcH', 'Course_Code__c

In [22]:
course_lookup_df = pd.DataFrame(course_lookup_list)
course_lookup_df

,ID_Course__c,Course_Code__c
0,a004x000003VQ9I,CIS-349
1,a004x000003VQ9S,CIS-438
2,a004x000003VQ98,BC-WEBDEV
3,a004x000003VQ93,BC-DATAVIZ
4,a004x000003VQ9X,CIS-430
5,a004x000003VQ9c,CIS-435
6,a004x000003VQ9N,CIS-405
7,a004x000003UMcR,CIS-438
8,a004x000003UMc2,BC-DATAVIZ
9,a004x000003UMcW,CIS-430


In [23]:
# Query the `Class` table from MySQL
query = '''
    SELECT 
        co.CourseCode,
        co.CourseName,
        cl.*
    FROM 
        class cl
        INNER JOIN course co
        ON cl.ID_Course = co.ID_Course

'''

class_data_df = pd.read_sql(query, conn)
class_data_df.head()

,CourseCode,CourseName,ID_Class,ID_Course,Section,StartDate,EndDate
0,BC-DATAVIZ,Data Visualization and Analytics,1,1,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14
1,BC-DATAVIZ,Data Visualization and Analytics,2,1,GWDC201805DATA3,2018-05-15,2018-11-08
2,BC-WEBDEV,Full Stack Web Development,3,2,GWARL201905WEB3,2019-05-14,2019-11-07
3,BC-UIUX,User Interface/User Experience,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07


In [24]:
class_data_df.rename(columns={
    'CourseCode':'Course_Code__c',
    'Section':'Section__c',
    'StartDate':'Start_Date__c',
    'EndDate':'End_Date__c',
    'CourseCode':'Course_Code__c'
}, inplace=True)


class_data_df = class_data_df[['Course_Code__c', 'Section__c', 'Start_Date__c', 'End_Date__c']]
class_data_df

,Course_Code__c,Section__c,Start_Date__c,End_Date__c
0,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14
1,BC-DATAVIZ,GWDC201805DATA3,2018-05-15,2018-11-08
2,BC-WEBDEV,GWARL201905WEB3,2019-05-14,2019-11-07
3,BC-UIUX,GWARL201905UIUX3,2019-05-14,2019-11-07


## Join the Class DataFrame with the Course lookup table
This join is necessary to successfully lookup the foreign key for the Course table 

In [25]:
class_data_df = pd.merge(class_data_df, course_lookup_df, how='left')
#class_data_df.drop(columns = ['ID_Class','ID_Course','CourseName','CreditHours','BootCampCourse','Course_Code__c'], inplace=True)

class_data_df.head()

,Course_Code__c,Section__c,Start_Date__c,End_Date__c,ID_Course__c
0,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14,a004x000003VQ93
1,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14,a004x000003UMc2
2,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14,a004x000003V7Nm
3,BC-DATAVIZ,GWDC201805DATA3,2018-05-15,2018-11-08,a004x000003VQ93
4,BC-DATAVIZ,GWDC201805DATA3,2018-05-15,2018-11-08,a004x000003UMc2


In [26]:
class_data_df
class_data_df['Start_Date__c'] = pd.to_datetime(class_data_df['Start_Date__c']).dt.date
class_data_df['End_Date__c'] = pd.to_datetime(class_data_df['End_Date__c']).dt.date

class_data_df.head()


,Course_Code__c,Section__c,Start_Date__c,End_Date__c,ID_Course__c
0,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14,a004x000003VQ93
1,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14,a004x000003UMc2
2,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14,a004x000003V7Nm
3,BC-DATAVIZ,GWDC201805DATA3,2018-05-15,2018-11-08,a004x000003VQ93
4,BC-DATAVIZ,GWDC201805DATA3,2018-05-15,2018-11-08,a004x000003UMc2


In [27]:
class_data_records = class_data_df.to_dict(orient='records')
class_data_records

[{'Course_Code__c': 'BC-DATAVIZ',
  'Section__c': 'GWU-ARL-DATA-PT-09-0',
  'Start_Date__c': datetime.date(2020, 9, 16),
  'End_Date__c': datetime.date(2020, 3, 14),
  'ID_Course__c': 'a004x000003VQ93'},
 {'Course_Code__c': 'BC-DATAVIZ',
  'Section__c': 'GWU-ARL-DATA-PT-09-0',
  'Start_Date__c': datetime.date(2020, 9, 16),
  'End_Date__c': datetime.date(2020, 3, 14),
  'ID_Course__c': 'a004x000003UMc2'},
 {'Course_Code__c': 'BC-DATAVIZ',
  'Section__c': 'GWU-ARL-DATA-PT-09-0',
  'Start_Date__c': datetime.date(2020, 9, 16),
  'End_Date__c': datetime.date(2020, 3, 14),
  'ID_Course__c': 'a004x000003V7Nm'},
 {'Course_Code__c': 'BC-DATAVIZ',
  'Section__c': 'GWDC201805DATA3',
  'Start_Date__c': datetime.date(2018, 5, 15),
  'End_Date__c': datetime.date(2018, 11, 8),
  'ID_Course__c': 'a004x000003VQ93'},
 {'Course_Code__c': 'BC-DATAVIZ',
  'Section__c': 'GWDC201805DATA3',
  'Start_Date__c': datetime.date(2018, 5, 15),
  'End_Date__c': datetime.date(2018, 11, 8),
  'ID_Course__c': 'a004x0000

## Insert `Class` Records into Salesforce

In [33]:
for rec in class_data_records:
 
    record = {
        'ID_Course__c': rec['ID_Course__c'],
        'Section__c': rec['Section__c'],
        'Start_Date__c': str(rec['Start_Date__c']),
        'End_Date__c': str(rec['End_Date__c']),
    }
    
    try:
        sf.Class__C.create(record)
    except Exception as e:
        print(e)

# Load Class Participant

In [68]:
sf_course_records = []

sf_course_data = sf.query('SELECT Name, Course_Code__c FROM Course__c')

for row in sf_course_data['records']:
    
    rec = {
        'ID_Course__c': row['Name'],
        'Course_Code__c': row['Course_Code__c']
    }
    
    sf_course_records.append(rec)
    
sf_course_df = pd.DataFrame(sf_course_records)
sf_course_df.head()

,ID_Course__c,Course_Code__c
0,a004x000003VQ9I,CIS-349
1,a004x000003VQ9S,CIS-438
2,a004x000003VQ98,BC-WEBDEV
3,a004x000003VQ93,BC-DATAVIZ
4,a004x000003VQ9X,CIS-430


In [69]:
sf_class_records = []

sf_class_data = sf.query('SELECT Name, ID_Course__c, Section__c FROM Class__c')

for row in sf_class_data['records']:
    
    rec = {
        'ID_Class__c': row['Name'],
        'ID_Course__c': row['ID_Course__c'],
        'Section__c': row['Section__c']
    }
    
    sf_class_records.append(rec)
 
sf_class_records

[{'ID_Class__c': 'a014x000008WKqQ',
  'ID_Course__c': 'a004x000003UMc2AAG',
  'Section__c': 'GWU-ARL-DATA-PT-09-0'},
 {'ID_Class__c': 'a014x000008WKqV',
  'ID_Course__c': 'a004x000003V7NmAAK',
  'Section__c': 'GWU-ARL-DATA-PT-09-0'},
 {'ID_Class__c': 'a014x000008WKr4',
  'ID_Course__c': 'a004x000003UMcCAAW',
  'Section__c': 'GWARL201905UIUX3'},
 {'ID_Class__c': 'a014x000008WKqf',
  'ID_Course__c': 'a004x000003UMc2AAG',
  'Section__c': 'GWDC201805DATA3'},
 {'ID_Class__c': 'a014x000008WKqL',
  'ID_Course__c': 'a004x000003VQ93AAG',
  'Section__c': 'GWU-ARL-DATA-PT-09-0'},
 {'ID_Class__c': 'a014x000008WKqk',
  'ID_Course__c': 'a004x000003V7NmAAK',
  'Section__c': 'GWDC201805DATA3'},
 {'ID_Class__c': 'a014x000008WKrE',
  'ID_Course__c': 'a004x000003VQ9DAAW',
  'Section__c': 'GWARL201905UIUX3'},
 {'ID_Class__c': 'a014x000008WKqz',
  'ID_Course__c': 'a004x000003V7NnAAK',
  'Section__c': 'GWARL201905WEB3'},
 {'ID_Class__c': 'a014x000008WKqp',
  'ID_Course__c': 'a004x000003VQ98AAG',
  'Section_

In [70]:
for rec in sf_class_records:
    rec['ID_Course__c'] = rec['ID_Course__c'][:-3]

In [71]:
sf_class_df = pd.DataFrame(sf_class_records)

#sf_class_df['ID_Course__c'] = sf_class_df['ID_Course__c'].apply(lambda id_course: id_course[:-3])

sf_class_df.head()

,ID_Class__c,ID_Course__c,Section__c
0,a014x000008WKqQ,a004x000003UMc2,GWU-ARL-DATA-PT-09-0
1,a014x000008WKqV,a004x000003V7Nm,GWU-ARL-DATA-PT-09-0
2,a014x000008WKr4,a004x000003UMcC,GWARL201905UIUX3
3,a014x000008WKqf,a004x000003UMc2,GWDC201805DATA3
4,a014x000008WKqL,a004x000003VQ93,GWU-ARL-DATA-PT-09-0


In [72]:
sf_class_xwalk = pd.merge(sf_class_df, sf_course_df, on='ID_Course__c')
sf_class_xwalk


#a044x000002rs51AAA
#a044x000002rs51

sf_class_xwalk

,ID_Class__c,ID_Course__c,Section__c,Course_Code__c
0,a014x000008WKqQ,a004x000003UMc2,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ
1,a014x000008WKqf,a004x000003UMc2,GWDC201805DATA3,BC-DATAVIZ
2,a014x000008WKqV,a004x000003V7Nm,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ
3,a014x000008WKqk,a004x000003V7Nm,GWDC201805DATA3,BC-DATAVIZ
4,a014x000008WKr4,a004x000003UMcC,GWARL201905UIUX3,BC-UIUX
5,a014x000008WKqL,a004x000003VQ93,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ
6,a014x000008WKqa,a004x000003VQ93,GWDC201805DATA3,BC-DATAVIZ
7,a014x000008WKrE,a004x000003VQ9D,GWARL201905UIUX3,BC-UIUX
8,a014x000008WKqz,a004x000003V7Nn,GWARL201905WEB3,BC-WEBDEV
9,a014x000008WKqp,a004x000003VQ98,GWARL201905WEB3,BC-WEBDEV


In [73]:
sf_student_records = []

sf_student_data = sf.query('SELECT Name, Student_ID__c FROM Student__c')

for row in sf_student_data['records']:
    
    rec = {
        'ID_Student__c': row['Name'],
        'Student_ID__c': row['Student_ID__c']
    }
    
    sf_student_records.append(rec)
    
sf_student_xwalk = pd.DataFrame(sf_student_records)
sf_student_xwalk.head()

,ID_Student__c,Student_ID__c
0,a024x000002hL5u,25005443
1,a024x000002hL51,25005602
2,a024x000002hL3y,25004961
3,a024x000002hL5G,25002714
4,a024x000002hL4D,25002589


In [74]:
 query = '''

    SELECT
        s.StudentID AS Student_ID__c,
        co.CourseCode AS Course_Code__c,
        cl.Section AS Section__c,
        cp.StartDate AS Start_Date__c,
        cp.EndDate AS End_Date__c
    FROM 
        classparticipant cp
        INNER JOIN class cl
        ON cp.ID_Class = cl.ID_Class
        INNER JOIN course co
        ON cl.ID_Course = co.ID_Course
        INNER JOIN student s
        ON cp.ID_Student = s.ID_Student

'''

classparticiant_df = pd.read_sql(query, conn)
classparticiant_df.head()

,Student_ID__c,Course_Code__c,Section__c,Start_Date__c,End_Date__c
0,25004961,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,None
1,25003514,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,None
2,25005833,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,None
3,25007334,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,None
4,25002589,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,None


In [60]:
#sf_class_xwalk
#sf_student_xwalk

class_participant_load = pd.merge(classparticiant_df, sf_class_xwalk, on=['Course_Code__c', 'Section__c'])

class_participant_load = pd.merge(class_participant_load, sf_student_xwalk)

class_participant_load = class_participant_load[['ID_Student__c','ID_Class__c','Start_Date__c','End_Date__c']]

class_participant_load['Start_Date__c'] = class_participant_load['Start_Date__c'].astype(str)

class_participant_load

class_participant_records = class_participant_load.to_dict(orient='records')
class_participant_records

[{'ID_Student__c': 'a024x000002hL3y',
  'ID_Class__c': 'a014x000008WKqQ',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Student__c': 'a024x000002hL3y',
  'ID_Class__c': 'a014x000008WKqV',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Student__c': 'a024x000002hL3y',
  'ID_Class__c': 'a014x000008WKqL',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Student__c': 'a024x000002hL43',
  'ID_Class__c': 'a014x000008WKqQ',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Student__c': 'a024x000002hL43',
  'ID_Class__c': 'a014x000008WKqV',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Student__c': 'a024x000002hL43',
  'ID_Class__c': 'a014x000008WKqL',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Student__c': 'a024x000002hL48',
  'ID_Class__c': 'a014x000008WKqQ',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Student__c': 'a024x000002hL48',
  'ID_Class__c': 'a014x000008WKqV',
  'Start_

In [75]:
for row in class_participant_records:
    try:
        sf.Class_Participant__c.create(row)
    except Exception as e:
        print(e)

# Load Staff Assignment

In [90]:
sf_staff_records = []
# NameError: name 'sf_Staff_data' is not defined
sf_staff_data = sf.query('SELECT Name, Last_Name__c, Middle_Name__c, First_Name__c, Birth_Date__c, Employee_ID__c FROM Staff__c')
for row in sf_staff_data['records']:
    rec = {
        'ID_Staff__c':row['Name'],
        'Last_Name__c': row['Last_Name__c'],
        'Middle_Name__c': row['Middle_Name__c'],
        'First_Name__c':row['First_Name__c'],
        'Birth_Date__c': row['Birth_Date__c'],
        'Employee_ID__c': row['Employee_ID__c']
    }
    sf_staff_records.append(rec)
sf_staff_records
sf_staff_xwalk = pd.DataFrame(sf_staff_records)
sf_staff_xwalk.head()

,ID_Staff__c,Last_Name__c,Middle_Name__c,First_Name__c,Birth_Date__c,Employee_ID__c
0,a044x000002rrpz,Rodney,Nicole,Heather,1994-06-07,000159108
1,a044x000002rrpu,Williams,De Angelo,Dartanion,1993-05-21,000160655
2,a044x000002rrpp,Sanford,Blair,Gemini,1992-04-22,000130109


In [91]:
query  ='''

select
    co.CourseCode,
    cl.Section,
    s.EmployeeID,
    sa.role, 
    sa.startdate,
    sa.enddate
from
    staffassignment as sa
    inner join staff s
    on sa.ID_Staff = s.ID_staff
    inner join class cl
    on sa.ID_Class=cl.ID_Class
    inner join course co
    on co.ID_Course=cl.ID_Course

'''
staffassignment_df = pd.read_sql(query, conn)
staffassignment_df.head()

,CourseCode,Section,EmployeeID,role,startdate,enddate
0,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000184220,Teacher Assistant,2020-09-16,None
1,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000159108,Teacher Assistant,2020-09-16,None
2,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000160655,Instructor,2020-09-16,None


In [92]:
print(staffassignment_df.columns)
print(sf_staff_xwalk.columns)
print(sf_class_xwalk.columns)

Index(['CourseCode', 'Section', 'EmployeeID', 'role', 'startdate', 'enddate'], dtype='object')
Index(['ID_Staff__c', 'Last_Name__c', 'Middle_Name__c', 'First_Name__c',
       'Birth_Date__c', 'Employee_ID__c'],
      dtype='object')
Index(['ID_Class__c', 'ID_Course__c', 'Section__c', 'Course_Code__c'], dtype='object')


In [108]:
staff_assignment_load_df = pd.merge(staffassignment_df, sf_class_xwalk, left_on=['CourseCode','Section'], right_on=['Course_Code__c','Section__c'])
staff_assignment_load_df.head()

,CourseCode,Section,EmployeeID,role,startdate,enddate,ID_Class__c,ID_Course__c,Section__c,Course_Code__c
0,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000184220,Teacher Assistant,2020-09-16,None,a014x000008WKqQ,a004x000003UMc2,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ
1,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000184220,Teacher Assistant,2020-09-16,None,a014x000008WKqV,a004x000003V7Nm,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ
2,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000184220,Teacher Assistant,2020-09-16,None,a014x000008WKqL,a004x000003VQ93,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ
3,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000159108,Teacher Assistant,2020-09-16,None,a014x000008WKqQ,a004x000003UMc2,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ
4,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000159108,Teacher Assistant,2020-09-16,None,a014x000008WKqV,a004x000003V7Nm,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ


In [109]:
staff_assignment_load_df = pd.merge(staff_assignment_load_df, sf_staff_xwalk, left_on='EmployeeID', right_on='Employee_ID__c')
staff_assignment_load_df.head()

,CourseCode,Section,EmployeeID,role,startdate,enddate,ID_Class__c,ID_Course__c,Section__c,Course_Code__c,ID_Staff__c,Last_Name__c,Middle_Name__c,First_Name__c,Birth_Date__c,Employee_ID__c
0,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000159108,Teacher Assistant,2020-09-16,None,a014x000008WKqQ,a004x000003UMc2,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ,a044x000002rrpz,Rodney,Nicole,Heather,1994-06-07,000159108
1,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000159108,Teacher Assistant,2020-09-16,None,a014x000008WKqV,a004x000003V7Nm,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ,a044x000002rrpz,Rodney,Nicole,Heather,1994-06-07,000159108
2,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000159108,Teacher Assistant,2020-09-16,None,a014x000008WKqL,a004x000003VQ93,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ,a044x000002rrpz,Rodney,Nicole,Heather,1994-06-07,000159108
3,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000160655,Instructor,2020-09-16,None,a014x000008WKqQ,a004x000003UMc2,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ,a044x000002rrpu,Williams,De Angelo,Dartanion,1993-05-21,000160655
4,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000160655,Instructor,2020-09-16,None,a014x000008WKqV,a004x000003V7Nm,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ,a044x000002rrpu,Williams,De Angelo,Dartanion,1993-05-21,000160655


In [110]:
staff_assignment_load_df = staff_assignment_load_df[['ID_Staff__c','ID_Class__c','role','startdate','enddate']]
staff_assignment_load_df

,ID_Staff__c,ID_Class__c,role,startdate,enddate
0,a044x000002rrpz,a014x000008WKqQ,Teacher Assistant,2020-09-16,None
1,a044x000002rrpz,a014x000008WKqV,Teacher Assistant,2020-09-16,None
2,a044x000002rrpz,a014x000008WKqL,Teacher Assistant,2020-09-16,None
3,a044x000002rrpu,a014x000008WKqQ,Instructor,2020-09-16,None
4,a044x000002rrpu,a014x000008WKqV,Instructor,2020-09-16,None
5,a044x000002rrpu,a014x000008WKqL,Instructor,2020-09-16,None


In [111]:
staff_assignment_load_df.rename(columns={
    'role':'Role__c',
    'startdate':'Start_Date__c',
    'enddate':'End_Date__c'
}, inplace=True)

staff_assignment_load_df.head()

,ID_Staff__c,ID_Class__c,Role__c,Start_Date__c,End_Date__c
0,a044x000002rrpz,a014x000008WKqQ,Teacher Assistant,2020-09-16,None
1,a044x000002rrpz,a014x000008WKqV,Teacher Assistant,2020-09-16,None
2,a044x000002rrpz,a014x000008WKqL,Teacher Assistant,2020-09-16,None
3,a044x000002rrpu,a014x000008WKqQ,Instructor,2020-09-16,None
4,a044x000002rrpu,a014x000008WKqV,Instructor,2020-09-16,None


In [112]:
staff_assignment_load_df['Start_Date__c'] = staff_assignment_load_df['Start_Date__c'].astype(str)
staff_assignment_load_df

,ID_Staff__c,ID_Class__c,Role__c,Start_Date__c,End_Date__c
0,a044x000002rrpz,a014x000008WKqQ,Teacher Assistant,2020-09-16,None
1,a044x000002rrpz,a014x000008WKqV,Teacher Assistant,2020-09-16,None
2,a044x000002rrpz,a014x000008WKqL,Teacher Assistant,2020-09-16,None
3,a044x000002rrpu,a014x000008WKqQ,Instructor,2020-09-16,None
4,a044x000002rrpu,a014x000008WKqV,Instructor,2020-09-16,None
5,a044x000002rrpu,a014x000008WKqL,Instructor,2020-09-16,None


In [113]:
staff_assignment_records = staff_assignment_load_df.to_dict(orient='records')
staff_assignment_records

[{'ID_Staff__c': 'a044x000002rrpz',
  'ID_Class__c': 'a014x000008WKqQ',
  'Role__c': 'Teacher Assistant',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Staff__c': 'a044x000002rrpz',
  'ID_Class__c': 'a014x000008WKqV',
  'Role__c': 'Teacher Assistant',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Staff__c': 'a044x000002rrpz',
  'ID_Class__c': 'a014x000008WKqL',
  'Role__c': 'Teacher Assistant',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Staff__c': 'a044x000002rrpu',
  'ID_Class__c': 'a014x000008WKqQ',
  'Role__c': 'Instructor',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Staff__c': 'a044x000002rrpu',
  'ID_Class__c': 'a014x000008WKqV',
  'Role__c': 'Instructor',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Staff__c': 'a044x000002rrpu',
  'ID_Class__c': 'a014x000008WKqL',
  'Role__c': 'Instructor',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None}]

In [115]:
for rec in staff_assignment_records:
    try:
        sf.Staff_Assignment__c.create(rec)
    except Exception as e:
        print(e)

## Example of Deleting Records

Select the IDs of the records first and then process the results.

Ultimately, you want a list of IDs in the end.


In [66]:
class_records = sf.query("SELECT Id FROM Class_Participant__c")
recs_to_delete = [{'Id': r['Id']} for r in class_records['records']]
recs_to_delete

[{'Id': 'a054x0000010elsAAA'},
 {'Id': 'a054x0000010em6AAA'},
 {'Id': 'a054x0000010emBAAQ'},
 {'Id': 'a054x0000010emGAAQ'},
 {'Id': 'a054x0000010emHAAQ'},
 {'Id': 'a054x0000010emLAAQ'},
 {'Id': 'a054x0000010emMAAQ'},
 {'Id': 'a054x0000010emQAAQ'},
 {'Id': 'a054x0000010emVAAQ'},
 {'Id': 'a054x0000010emaAAA'},
 {'Id': 'a054x0000010emfAAA'},
 {'Id': 'a054x0000010emkAAA'},
 {'Id': 'a054x0000010empAAA'},
 {'Id': 'a054x0000010emuAAA'},
 {'Id': 'a054x0000010emzAAA'},
 {'Id': 'a054x0000010en4AAA'},
 {'Id': 'a054x0000010en9AAA'},
 {'Id': 'a054x0000010enEAAQ'},
 {'Id': 'a054x0000010enJAAQ'},
 {'Id': 'a054x0000010enKAAQ'},
 {'Id': 'a054x0000010enOAAQ'},
 {'Id': 'a054x0000010enPAAQ'},
 {'Id': 'a054x0000010enTAAQ'},
 {'Id': 'a054x0000010enYAAQ'},
 {'Id': 'a054x0000010endAAA'},
 {'Id': 'a054x0000010eniAAA'},
 {'Id': 'a054x0000010ennAAA'},
 {'Id': 'a054x0000010ensAAA'},
 {'Id': 'a054x0000010enxAAA'},
 {'Id': 'a054x0000010eo2AAA'},
 {'Id': 'a054x0000010eo7AAA'},
 {'Id': 'a054x0000010eoCAAQ'},
 {'Id': 

In [ ]:
#sf.bulk.Course__c.delete(recs_to_delete)

In [67]:
for rec in recs_to_delete:
    try:
        sf.Class_Participant__c.delete(rec['Id'])
    except Exception as e:
        print(e)